In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
def db_query(query):
    conn = sql.connect('gtfs/gtfs_new.db')
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

Query for table to self-compare:

``` sql
SELECT stop_name, location, stop_id, route_short_name
FROM
(SELECT route_stop.stop_id, stop_name, location, route_short_name
FROM
(SELECT distinct stop_id, route_short_name
FROM
(SELECT trips.route_id, trip_id, route_short_name FROM trips
INNER JOIN routes ON trips.route_id = routes.route_id) as trip_route
INNER JOIN stop_times ON trip_route.trip_id = stop_times.trip_id) as route_stop
INNER JOIN stops ON route_stop.stop_id = stops.stop_id)
```

In [3]:
query = '''
SELECT a.stop_name as StopName_A,
b.stop_name as StopName_B,
a.location as Location_A,
b.location as Location_B,
a.stop_id as StopID_A,
b.stop_id as StopID_B,
a.route_short_name as Route_A,
b.route_short_name as Route_B,
a.route_color as RouteColor_A,
b.route_color as RouteColor_B
FROM
(SELECT route_stop.stop_id, stop_name, location, route_short_name, route_color
FROM
(SELECT distinct stop_id, route_short_name, route_color
FROM
(SELECT trips.route_id, trip_id, route_short_name, route_color FROM trips
INNER JOIN routes ON trips.route_id = routes.route_id) as trip_route
INNER JOIN stop_times ON trip_route.trip_id = stop_times.trip_id) as route_stop
INNER JOIN stops ON route_stop.stop_id = stops.stop_id) as a
INNER JOIN
(SELECT route_stop.stop_id, stop_name, location, route_short_name, route_color
FROM
(SELECT distinct stop_id, route_short_name, route_color
FROM
(SELECT trips.route_id, trip_id, route_short_name, route_color FROM trips
INNER JOIN routes ON trips.route_id = routes.route_id) as trip_route
INNER JOIN stop_times ON trip_route.trip_id = stop_times.trip_id) as route_stop
INNER JOIN stops ON route_stop.stop_id = stops.stop_id) as b
ON (a.stop_id < b.stop_id and a.stop_name = b.stop_name and a.location < b.location and a.route_short_name < b.route_short_name)
'''

df = db_query(query)

In [5]:
df.to_csv('stop_pairs.csv', index=False)

In [4]:
df

,StopName_A,StopName_B,Location_A,Location_B,StopID_A,StopID_B,Route_A,Route_B,RouteColor_A,RouteColor_B
0,Deák Ferenc tér,Deák Ferenc tér,"(47.497413, 19.055158)","(47.497593, 19.053857)",007884,F00961,47,M2,FFD800,E41F18
1,Deák Ferenc tér,Deák Ferenc tér,"(47.497413, 19.055158)","(47.498843, 19.053605)",007884,F00960,47,M2,FFD800,E41F18
2,Deák Ferenc tér,Deák Ferenc tér,"(47.497413, 19.055158)","(47.49772, 19.05367)",007884,F00962,47,M1,FFD800,FFD800
3,Deák Ferenc tér,Deák Ferenc tér,"(47.497413, 19.055158)","(47.497728, 19.054142)",007884,F00963,47,M1,FFD800,FFD800
4,Deák Ferenc tér,Deák Ferenc tér,"(47.497413, 19.055158)","(47.49744, 19.055729)",007884,009309,47,934,FFD800,000000
...,...,...,...,...,...,...,...,...,...,...
25436,Batthyány tér,Batthyány tér,"(47.506776, 19.039318)","(47.506826, 19.038103)",F00060,F00063,H5,M2,8A236C,E41F18
25437,Batthyány tér,Batthyány tér,"(47.506776, 19.039318)","(47.507412, 19.036739)",F00060,F00062,H5,M2,8A236C,E41F18
25438,"Margit híd, budai hídfő","Margit híd, budai hídfő","(47.514908, 19.039227)","(47.515041, 19.038937)",F00186,F00236,H5,N19,8A236C,FF9900
25439,Batthyány tér,Batthyány tér,"(47.506776, 19.039318)","(47.506826, 19.038103)",003209,F00063,H5,M2,8A236C,E41F18
